In [1]:
import pandas as pd

# Read the table
# The table headers are in row 0
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

# Create the initial dataframe from the table
df = pd.DataFrame(data = table[0])

#number of rows and columns
print(df.shape)
# Sample output of the Table
df.head()


(287, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront



# Handle rows where Borough is set but Neighbourhood is Not assigned

### Some of the rows have Borough set but Neighbourhood is Not assigned. Setting Neighbourhood the same as the Borough value.

In [2]:
df[(df.Borough != 'Not assigned') & (df.Neighbourhood == 'Not assigned')]

,Postcode,Borough,Neighbourhood
7,M7A,Queen's Park,Not assigned


Only one row ! Will fix manually

In [3]:
df.loc[df.Borough == "Queen's Park", 'Neighbourhood'] = "Queen's Park"

In [4]:
df[(df.Borough != 'Not assigned') & (df.Neighbourhood == 'Not assigned')]

,Postcode,Borough,Neighbourhood


This means it is fixed

# Remove rows where Borough & Neighbourhood are Not assigned

In [5]:
df = df[(df.Borough != 'Not assigned') | (df.Neighbourhood != 'Not assigned')]

Now checking our table again

In [6]:
#number of rows and columns
print(df.shape)
# Sample output of the Table
df.head()

(210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


# Group by Postal Code and Borough

In [7]:
new_d = pd.DataFrame(df.groupby(
    ['Postcode', 'Borough'])['Neighbourhood'].apply(
    lambda x: ', '.join(x))).reset_index()

In [8]:
new_d.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


checking last 5 rows as well

In [9]:
new_d.tail()

,Postcode,Borough,Neighbourhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


In [10]:
new_d.shape

(103, 3)

# Get Latitude & Longitude for each Postal Code

In [11]:
import geocoder

In [12]:
def get_geocoder(postal_code_from_df):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df.strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude
   

In [15]:
new_d['Latitude'], new_d['Longitude'] = zip(*new_d['Postcode'].apply(get_geocoder))
new_d.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [16]:
new_d

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
...,...,...,...,...,...
98,M9N,York,Weston,43.704845,-79.517546
99,M9P,Etobicoke,Westmount,43.696505,-79.530252
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.686810,-79.557284
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.743145,-79.584664
